## Limpieza de textos

Vamos a eliminar genero y carácteres redundantes de la columna descripcion, además de preprocesar y tokenizar el texto para facilitar la lectura al modelo

Cambios:

 Eliminar URL (realizando antes)<br>
 Eliminar números (realizando antes)<br>
 Eliminar correos (realizando antes) <br> <br>
 Traducción de textos en otras lenguas al castellano. <br>
 Convertir a minúsculas<br>
 Eliminar signos de puntuación<br>
 Lematizar texto<br>
 Detectar y eliminar patrones tipo /a, /as

## Análisis de Sentimiento

El objetivo de esta sección es analizar el sentimiento del texto contenido en la columna descripcion utilizando distintos modelos de *sentiment analysis*.  
Cada modelo devuelve una puntuación de probabilidad para tres clases posibles:  
- negativo (`neg`)  
- neutral (`neu`)  
- positivo (`pos`)

Posteriormente, se combinan los resultados mediante un **voting classifier ponderado**, donde los modelos más robustos tienen mayor influencia en la clasificación final.

---

### Modelos utilizados

1. **VADER (NLTK)**  
   Modelo léxico basado en reglas, ligero y eficiente, ideal para análisis rápidos de texto.  
   Devuelve directamente las tres probabilidades `neg`, `neu` y `pos`.

2. **PySentimiento**  
   Implementa un modelo basado en *BETO* (BERT entrenado en español) optimizado para análisis de sentimientos.  
   Ofrece un mejor desempeño en textos en español que `TextBlob`, que estaba pensado para inglés y fue nuestra primera opción a usar.

3. **BERT (BETO de HuggingFace)**  
   Modelo de *transformers* profundo, más costoso computacionalmente pero más preciso.  
   Se usa el modelo público [`finiteautomata/beto-sentiment-analysis`](https://huggingface.co/finiteautomata/beto-sentiment-analysis), que devuelve las probabilidades de cada clase.  
   Para este modelo se le asigna mayor peso dentro del ensemble.



In [1]:
!pip install pandas nltk pysentimiento transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 25.1 MB/s eta 0:00:00


In [30]:
import pandas as pd
import torch
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from transformers import pipeline

nltk.download('vader_lexicon') # descargar el modelo para usar el sentiment analysis

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [33]:
# VADER
sia = SentimentIntensityAnalyzer()

def vader_scores(text: str):
    s = sia.polarity_scores(text)
    return {"neg": float(s["neg"]), "neu": float(s["neu"]), "pos": float(s["pos"])}

# TEXTBLOB
def textblob_scores(text: str):
    p = TextBlob(text).sentiment.polarity
    if p > 0:
        return {"neg": 0.0, "neu": 1 - p, "pos": p}
    elif p < 0:
        return {"neg": abs(p), "neu": 1 - abs(p), "pos": 0.0}
    else:
        return {"neg": 0.05, "neu": 0.9, "pos": 0.05}

# BETO (BERT español)
device = 0 if torch.cuda.is_available() else -1
bert_pipeline = pipeline(
    "sentiment-analysis",
    model="finiteautomata/beto-sentiment-analysis",
    top_k=None,
    device=device
)

def bert_scores(text: str):
    # Truncar texto a 512 tokens
    tokenizer = bert_pipeline.tokenizer
    max_seq_length = 512
    tokens = tokenizer.encode(text, truncation=True, max_length=max_seq_length, return_tensors="pt")
    decoded_text = tokenizer.decode(tokens[0], skip_special_tokens=True)

    # Ejecutar el pipeline
    result = bert_pipeline(decoded_text, top_k=None)

    # Normalizar formato de salida
    if isinstance(result[0], list):
        result = result[0]
    scores = {r["label"].lower(): float(r["score"]) for r in result}
    return {
        "neg": scores.get("neg", 0.0),
        "neu": scores.get("neu", 0.0),
        "pos": scores.get("pos", 0.0)
    }

# ENSEMBLE PONDERADO
def weighted_sentiment(text: str, weights=(0.3, 0.3, 0.4)):
    vader_w, blob_w, bert_w = weights

    v = vader_scores(text)
    t = textblob_scores(text)
    b = bert_scores(text)

    combined = {
        "neg": vader_w * v["neg"] + blob_w * t["neg"] + bert_w * b["neg"],
        "neu": vader_w * v["neu"] + blob_w * t["neu"] + bert_w * b["neu"],
        "pos": vader_w * v["pos"] + blob_w * t["pos"] + bert_w * b["pos"]
    }

    final_label = max(combined, key=combined.get)
    label_map = {"neg": -1, "neu": 0, "pos": 1}
    return label_map[final_label]

In [34]:
df = pd.read_csv("/EURES_CATEGORIZADO.csv", sep=",")
df["sentimiento"] = df["descripcion"].apply(weighted_sentiment)



In [35]:
df.head()

,id,timestamp,titulo,ocupacion,descripcion,provincia,tipo_contrato,descripcion_proc,sector,probs,sentimiento
0,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,AGENTE COMERCIAL DE SEGUROS (REF.: 6891),corredor de seguros/corredora de seguros,tareas:prospección de nuevos asegurados.planif...,Asturias,Contrato,tarea prospección asegurado planificación gest...,Administración y Finanzas,{'Hostelería y Turismo': np.float64(0.04141047...,0
1,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,PERSONAL CONDUCCIÓN DE CAMIONES RÍGIDOS Y GÓND...,Conductor de vehículo de carga/conductora de v...,descripción: se necesita cubrir cuatro puestos...,Huesca,Contrato,descripción necesitar cubrir puesto empresa mo...,Logística y Transporte,{'Hostelería y Turismo': np.float64(0.03098547...,0
2,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,EDUCADORES SOCIALES,Trabajador social/trabajadora social,educador social para hogar en arinaga. fines d...,Las Palmas,Determinado,educador social hogar arinaga fin semana festi...,Educación y Formación,{'Hostelería y Turismo': np.float64(0.09141705...,0
3,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,PIZZERO (REF. 042025002051),Pizzero/pizzera,funciones: elaboración de pizzas requisitos: 2...,Islas Baleares,Determinado,función elaboración pizza requisito mes experi...,Hostelería y Turismo,{'Hostelería y Turismo': np.float64(0.32179939...,0
4,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,INTÉRPRETES DE LA LENGUA DE SIGNOS,intérprete de lengua de signos,intérprete de lengua de signos para puestos en...,Santa Cruz de Tenerife,Determinado,intérprete lengua signo puesto ies manuel mart...,"Cultura, Arte y Ocio",{'Hostelería y Turismo': np.float64(0.08064752...,0


In [37]:
df.to_csv("EURES_CAT_SENTIMENT.csv", index=False)

In [36]:
df["sentimiento"].value_counts()

,count
sentimiento,
0,7236
1,24
